In [1]:
import nadaraya
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nadaraya import NadarayaKernel
from tqdm import tqdm

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('train.csv')
data

,id,Row#,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield
0,0,639.0,25.0,0.50,0.25,0.75,0.63,94.6,57.2,79.0,68.2,33.0,55.9,34.0,0.56,0.402948,0.409261,31.274591,4418.44126
1,1,124.0,12.5,0.25,0.25,0.25,0.25,77.4,46.8,64.7,55.8,27.0,45.8,34.0,0.56,0.500438,0.445494,34.467567,5862.80545
2,2,485.0,25.0,0.50,0.25,0.38,0.75,94.6,57.2,79.0,68.2,33.0,55.9,24.0,0.39,0.509001,0.459421,36.624966,6079.08526
3,3,324.0,12.5,0.25,0.25,0.75,0.75,94.6,57.2,79.0,68.2,33.0,55.9,16.0,0.26,0.583379,0.498056,40.865478,7400.77538
4,4,235.0,12.5,0.25,0.25,0.50,0.63,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.447669,0.423764,33.298861,4858.24073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,258.0,12.5,0.25,0.25,0.50,0.63,69.7,42.1,58.2,50.2,24.3,41.2,34.0,0.56,0.335927,0.352186,28.793440,3182.69865
14996,14996,256.0,12.5,0.25,0.25,0.38,0.63,77.4,46.8,64.7,55.8,27.0,45.8,34.0,0.56,0.545095,0.470719,38.042756,5862.80545
14997,14997,324.0,12.5,0.25,0.25,0.75,0.50,77.4,46.8,64.7,55.8,27.0,45.8,16.0,0.26,0.481801,0.442535,35.414082,5569.13925
14998,14998,691.0,25.0,0.50,0.38,0.38,0.75,86.0,52.0,71.9,62.0,30.0,50.8,1.0,0.10,0.552359,0.470847,38.625436,6795.88187


In [3]:
data.drop(['id', 'Row#'], axis=1, inplace=True)

In [4]:
train_x = data.drop(['yield'], axis=1)
train_y = data['yield']

In [5]:
kernel = NadarayaKernel(bandwidth=np.identity(data.shape[1]-1))
kernel.fit(data)

In [6]:
train = data[:int(0.8*len(data))]
test = data[int(0.8*len(data)):]

In [16]:
kernel.fit(data)

In [8]:
test_answers = test['yield']

In [9]:
test = test.drop(['yield'], axis=1)

In [10]:
function = 0
for i in tqdm(range(len(test))):
    function += abs(kernel.evaluate(np.array(test.iloc[i]))- test_answers.iloc[i])

  0%|          | 0/3000 [00:00<?, ?it/s]

100%|██████████| 3000/3000 [04:51<00:00, 10.31it/s]


In [11]:
testing = pd.read_csv('test.csv')
test_ids = testing['id']
testing.drop(['id', 'Row#'], axis=1, inplace=True)

In [17]:
submission = pd.DataFrame(columns=['id', 'yield'])
submission['id'] = test_ids
for i in tqdm(range(len(testing))):
    submission.loc[i, 'yield'] = kernel.evaluate(np.array(testing.iloc[i]))

100%|██████████| 10000/10000 [14:56<00:00, 11.16it/s]


In [13]:
function/len(test)

350.91049099670374

In [19]:
submission.to_csv('submission_1.csv', index=False)